**State Boundaries Cleaning**

Using tl_2024_us_state data, let's filter down to desired columns, as well as the desired state. This process can be expanded in the future to include multiple states.

In [1]:
import geopandas as gpd
import sys
from pathlib import Path

notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.append(str(project_root))


from scripts import config

In [3]:
gdf = gpd.read_file(config.RAW_DATA_PATH / "tl_2024_us_state" / "tl_2024_us_state.shp")


print(gdf.columns.tolist())
print(gdf.loc[0])

['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'GEOIDFQ', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']
REGION                                                      3
DIVISION                                                    5
STATEFP                                                    54
STATENS                                              01779805
GEOID                                                      54
GEOIDFQ                                           0400000US54
STUSPS                                                     WV
NAME                                            West Virginia
LSAD                                                       00
MTFCC                                                   G4000
FUNCSTAT                                                    A
ALAND                                             62266513826
AWATER                                              488918898
INTPTLAT                   

First, let's filter on the desired state — for now, we're sticking to Colorado.

In [5]:
gdf_co = gdf[gdf['NAME'] == 'Colorado']
gdf_co.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,GEOIDFQ,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
26,4,8,08,01779779,08,0400000US08,CO,Colorado,00,G4000,A,268418973518,1185541418,+38.9937669,-105.5087122,"POLYGON ((-105.15504 36.99526, -105.15543 36.9..."


Next, let's keep only the desired columns. All we really need for now is geometry and name.

In [6]:
gdf_cleaned = gdf_co[['geometry', 'NAME']]
gdf_cleaned.head()

,geometry,NAME
26,"POLYGON ((-105.15504 36.99526, -105.15543 36.9...",Colorado


**Buffering**

When generating a legality layer for a given state, we must take into account distance from features like water and MVUM roads. If you're near the edge of the given state boundary, it's possible that edge cases may arise where water or roads just outside the border aren't accounted for, leading to an inaccurate layer. Let's buffer the state boundary by 300 ft (the minimum distance from MVUM roads for legal camping) to account for these edge cases.

Before buffering, let's project the data to a common CRS.


In [7]:
gdf_projected = gdf_cleaned.to_crs(config.BUFFER_CRS)

Now, buffer!

In [10]:
gdf_buffered = gdf_projected.copy()
gdf_buffered = gdf_buffered.set_geometry(gdf_buffered.geometry.buffer(config.BUFFER_DISTANCE_ROAD_METERS, cap_style="round"))

Now, let's export to the output folder.

In [ ]:
output_folder = config.OUTPUT_DIR / "state_boundary_buffered"
output_folder.mkdir(parents=True, exist_ok=True)

output_fp = output_folder / "state_boundary_buffered_co.gpkg"
gdf_buffered.to_file(output_fp, driver="GPKG")

print("Saved buffered state boundary to: ", output_fp)